# 네이버 날씨 정보 스크래핑 하기

In [ ]:
import datetime
from bs4 import BeautifulSoup as bs
import requests as req

res = req.get("https://search.naver.com/search.naver?query=날씨")
html = res.text
soup = bs(html, "html.parser")

location = soup.find("div", {"class": "title_area _area_panel"}).find('h2', {'class':'title'}).text
weather = soup.find("div", attrs={'class':'weather_main'}).text
temperature = soup.find("div", attrs={'class':'temperature_text'}).text
lowest = soup.find('span',{'class':'lowest'}).text
highest = soup.find('span', {'class':'highest'}).text
dust = soup.find('li',{'class':'item_today level2'}).text

while True:
    print()
    print("=" * 30)
    print("날씨 확인 프로그램")
    print("=" * 30)
    print("접속 시간:", datetime.datetime.now())
    print("접속 위치:", location.lstrip())
    print("-" * 30)
    print("1. 현재 날씨 및 온도 확인")
    print("2. 최저/최고 온도 확인")
    print("3. 미세먼지 확인")
    print("0. 종료")
    print("-" * 30)
    try:
        sel = int(input("번호를 입력하세요."))
        print()
        if sel == 0:
            print("프로그램을 종료합니다.")
            break
        elif sel == 1:
            print("=" * 30)
            print("현재 날씨 확인")
            print("=" * 30)
            print("-" * 30)
            print(weather,'/',temperature)
            print("-" * 30)
        elif sel == 2:
            print("=" * 30)
            print("최저/최고 기온 확인")
            print("=" * 30)
            print("-" * 30)
            print("최저 기온:", lowest[4:])
            print("최고 기온:", highest[4:])
            print("-" * 30)
        elif sel == 3:
            print("=" * 30)
            print("미세먼지 확인")
            print("=" * 30)
            print("-" * 30)
            print(dust.strip()[5:])
        else:
            print("다시 입력해주세요.")
    except:
        print("오류 발생.")



# 네이버에서 bts이미지를 검색한 후 나온 결과 중 5개의 이미지를 저장하기

In [ ]:
import os
import time

import requests
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions

url ='https://search.naver.com/search.naver?where=image&sm=tab_jum&query=bts'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
driver.get(url)
time.sleep(1)

soup = BeautifulSoup(driver.page_source)
bts_imgs = soup.find_all('img', attrs={'class' : '_image _listImage'})

print('그림의 개수 :', len(bts_imgs)) # 첫 화면에 보이는 이미지의 개수=> 더 많음

# 이미지 저장할 폴더 생성
img_dir = './bts_img/'

if not os.path.exists(img_dir) : # 디렉토리가 없으면 생성
    os.makedirs(img_dir) 
    print('폴더 생성')
else : 
    print('폴더가 존재함')
    
for i, image in enumerate(bts_imgs) :
    img_url = image['src']
    
    if(not img_url.startswith('http')) : continue
    
    img_src = requests.get(img_url, headers=headers)
    img_src.raise_for_status()
    
    with open(img_dir + 'bts_{}.jpg'.format(i), 'wb') as f:
        print(img_url)
        f.write(img_src.content)
        if i >= 10 : break;
        
print('BTS 사진 저장 완료')    

# 멜론 사이트 최신곡 검색

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = 'https://www.melon.com/new/index.htm'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
response = requests.get(url, headers=headers)
bs = BeautifulSoup(response.text, 'html.parser')

song_list = bs.find_all("tr")
titles = []
singers = []
albums = []
count = 0
for rank, song in enumerate(song_list, 1):
    # class = "wrap_song_info" 속성을 갖는 테그가 2개여서 find_all로 검색
    song_info = song.find_all('div',attrs={'class':'wrap_song_info'})
    #print(len(song_info))
    if not song_info: # song_info 값이 None인 경우 skip (노래 정보를 갖고 있는 tr 태그가 아닌경우)
        continue
    
    count += 1
    if count > 10: break
        
    titles.append(song_info[0].find('div',attrs={'class':'ellipsis rank01'}).find('a').text)
    singers.append(song_info[0].find('div',attrs={'class':'ellipsis rank02'}).find('a').text)
    albums.append(song_info[1].find('div',attrs={'class':'ellipsis rank03'}).find('a').text)
    
music_rank_df = pd.DataFrame({'곡목':titles, '가수':singers, '앨범':albums}, index=np.arange(1,11))
music_rank_df

# 대문자 시작 단어 찾기

In [ ]:
import re

s = 'Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with Tests mode.'

m = re.findall('[A-Z]\w+', s)
print(m)

# 이메일 주소 추출하기

In [ ]:
import re

s = 'Ryan has sent an invoice email to john.d@yahoo.com by using his email id ryan.arjun@gmail.com and he also shared a copy to his boss rosy.gray@amazon.co.uk on the cc part.'

p = re.compile('[a-zA-Z]\w*[.]?\w*@[a-zA-Z]\w*[.]\w+[.]?\w*')
m = p.findall(s)
print(m)